# SIMULATOR STARTER

In [1]:
import sys
!{sys.executable} -m pip install psycopg2

/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:19: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import psycopg2
from psycopg2.extras import execute_values
import time
import requests
import numpy as np
import pandas as pd
import statistics
from scipy import stats, signal
from sklearn.model_selection import train_test_split
from scipy.stats import ttest_ind

/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


# Conection and create table

In [3]:
def connect_postgress():
    """ open conexion to the PostgreSQL database"""
    conn = psycopg2.connect(
        host="postgres-data",
        port=5432,
        database="sc",
        user="postgres",
        password="postgres")
    return conn

def create_tables():
    """ create tables in the PostgreSQL database"""
    command = (
        """
        CREATE TABLE profiles (
            TIME FLOAT,
            MACHINE BIGINT,
            DATA VARCHAR(10000),
            MAX FLOAT,
            MEAN FLOAT,
            MIN FLOAT
        );
        CREATE TABLE p_value (
            TIME FLOAT,
            MACHINE BIGINT,
            PVALUE FLOAT
        )
        """)
    conn = None
    a = False
    try:
        # read the connection parameters
        #params = config()
        # connect to the PostgreSQL server
        #conn = psycopg2.connect(**params)
        conn = connect_postgress()
        cur = conn.cursor()
        # create table one by one
        #for command in commands:
        cur.execute(command)
        # close communication with the PostgreSQL database server
        cur.close()
        # commit the changes
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        a = True
    finally:
        if conn is not None:
            conn.close()
    return a

def drop_tables():
    """ create tables in the PostgreSQL database"""
    command = (
        """
        DROP TABLE IF EXISTS profiles;
        DROP TABLE IF EXISTS p_value
        """)
    conn = None
    a = False
    try:
        # read the connection parameters
        #params = config()
        # connect to the PostgreSQL server
        #conn = psycopg2.connect(**params)
        conn = connect_postgress()
        cur = conn.cursor()
        # create table one by one
        #for command in commands:
        cur.execute(command)
        # close communication with the PostgreSQL database server
        cur.close()
        # commit the changes
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        a = True
    finally:
        if conn is not None:
            conn.close()
    return a


def insert_values(table_name,values,sql):
    """ insert a new vendor into the vendors table """
    conn = None
    vendor_id = None
    print(sql)
    try:
        # read database configuration
        # params = config()
        # connect to the PostgreSQL database
        conn = connect_postgress()
        #conn = psycopg2.connect(**params)
        # create a new cursor
        cur = conn.cursor()
        # execute the INSERT statement
        # cur.execute(sql, (values,))
        # execute the SQL statement
        execute_values(cur, sql, values)

        # get the generated id back
        #cur.fetchone()
        # cur.fetchall()
        # commit the changes to the database
        conn.commit()
        # close communication with the database
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        id_values = None
    finally:
        if conn is not None:
            conn.close()

### Run simulator

In [5]:
def run_simulator(num_machines = 1,step = 0.001, sleep = 2, cycles = 2000, drop = True):
    if drop:
        a = drop_tables()
    a = create_tables()
    
    n = 0
    base = 365.25*24*60*60
    timestamp = 0
    headers = {'Content-Type': "application/json", 'Accept': "application/json"}
    
    #START P_VALUE
    table_name = "p_value"
    values = [("0","0","1"),("0","1","1"),("0","2","1")]
    sql = """INSERT INTO """+table_name+""" (TIME,MACHINE,PVALUE) VALUES %s """
    insert_values(table_name,values,sql)
    
    while n != cycles:
        print("timestamp: ",timestamp)
        json = str(int(timestamp))
        for machine in range(num_machines):
            r = requests.get('http://172.16.56.53:5001/get_curve_time',json=json,headers=headers)
            values = r.json()
            max_val, mean_val, min_val = max(values), np.mean(values), min(values)
            value = [(str(timestamp), 
                      str(machine), 
                      str(values), 
                      str(max_val), 
                      str(mean_val), 
                      str(min_val))]
            table_name = "profiles"
            sql = """INSERT INTO """+table_name+""" (TIME,MACHINE,DATA,MAX,MEAN,MIN) VALUES %s """
            insert_values(table_name,value,sql)
        time.sleep(sleep)
        timestamp = timestamp + base * step
        n = n + 1

In [ ]:
run_simulator(num_machines = 3)

INSERT INTO p_value (TIME,MACHINE,PVALUE) VALUES %s 
timestamp:  0
INSERT INTO profiles (TIME,MACHINE,DATA,MAX,MEAN,MIN) VALUES %s 
INSERT INTO profiles (TIME,MACHINE,DATA,MAX,MEAN,MIN) VALUES %s 
INSERT INTO profiles (TIME,MACHINE,DATA,MAX,MEAN,MIN) VALUES %s 
timestamp:  31557.600000000002
INSERT INTO profiles (TIME,MACHINE,DATA,MAX,MEAN,MIN) VALUES %s 
INSERT INTO profiles (TIME,MACHINE,DATA,MAX,MEAN,MIN) VALUES %s 
INSERT INTO profiles (TIME,MACHINE,DATA,MAX,MEAN,MIN) VALUES %s 
timestamp:  63115.200000000004
INSERT INTO profiles (TIME,MACHINE,DATA,MAX,MEAN,MIN) VALUES %s 
INSERT INTO profiles (TIME,MACHINE,DATA,MAX,MEAN,MIN) VALUES %s 
INSERT INTO profiles (TIME,MACHINE,DATA,MAX,MEAN,MIN) VALUES %s 
timestamp:  94672.8
INSERT INTO profiles (TIME,MACHINE,DATA,MAX,MEAN,MIN) VALUES %s 
INSERT INTO profiles (TIME,MACHINE,DATA,MAX,MEAN,MIN) VALUES %s 
INSERT INTO profiles (TIME,MACHINE,DATA,MAX,MEAN,MIN) VALUES %s 
timestamp:  126230.40000000001
INSERT INTO profiles (TIME,MACHINE,DATA,